In [4]:
#Extractive Summary method
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
import heapq

nltk.download('punkt')
nltk.download('stopwords')

def simple_summarize(text, num_sentences=2):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize words and remove stopwords
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]

    # Calculate word frequencies
    freq = FreqDist(words)

    # Score sentences based on word frequencies
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in freq:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = freq[word]
                else:
                    sentence_scores[sentence] += freq[word]

    # Get the top N sentences
    summary_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)

    # Join the top sentences
    summary = ' '.join(summary_sentences)

    return summary

# The text to summarize
text = """Artificial intelligence refers to the simulation of human intelligence in machines
that are programmed to think like humans and mimic their actions. AI is being
applied in a wide range of industries, from healthcare to finance, and has the
potential to improve efficiency and decision-making."""

# Generate the summary
summary = simple_summarize(text)
print("Extractive Summary:", summary)

Extractive Summary: Artificial intelligence refers to the simulation of human intelligence in machines
that are programmed to think like humans and mimic their actions. AI is being
applied in a wide range of industries, from healthcare to finance, and has the
potential to improve efficiency and decision-making.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Abstractive Summary method
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

def abstractive_summarize(text, max_length=60, min_length=10):
    # Load pre-trained model and tokenizer
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

    # Encode the text
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'],
                                 num_beams=4,
                                 max_length=max_length,
                                 min_length=min_length,
                                 length_penalty=2.0,
                                 early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# The text to summarize
text = """Artificial intelligence refers to the simulation of human intelligence in machines
that are programmed to think like humans and mimic their actions. AI is being
applied in a wide range of industries, from healthcare to finance, and has the
potential to improve efficiency and decision-making."""

# Generate the summary
summary = abstractive_summarize(text)
print("Abstractive Summary:", summary)

Abstractive Summary: Artificial intelligence refers to the simulation of human intelligence in machinesthat are programmed to think like humans and mimic their actions. AI is being applied in a wide range of industries, from healthcare to finance, and has the potential to improve efficiency and decision-making.
